In [ ]:
##### Copyright 2025 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# MatFormer Lab:

 <table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/[Gemma_3n]MatFormer_Lab.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>


Gemma 3n is a multimodal, multilingual model from the Gemma family of models. You can read about Gemma 3n in [its docs](https://ai.google.dev/gemma/docs/gemma-3n) and [launch blog post](https://developers.googleblog.com/en/introducing-gemma-3n-developer-guide). It is a unique model that is natively elastic, which means you have nested models! Gemma 3n was trained as a E4B model (effectively loads 4B parameters) with 35 layers and a 16,384 FFN hidden dimension. It has a E2B (30 layers and 8,192 FFN hidden dimension) nested within it and jointly trained as a MatFormer.

[MatFormer](https://arxiv.org/abs/2310.07707) (🪆Matryoshka Transformer) architecture is a novel nested transformer built for elastic inference. Think of it like Matryoshka dolls: a larger model contains smaller, fully functional versions of itself. This approach extends the concept of [Matryoshka Representation Learning](https://huggingface.co/papers/2205.13147) from just embeddings to all transformer components.

Saving memory by having E2B nested within E4B is practically useful, but what makes MatFormer so powerful is its capability to smoothly span the entire Pareto-optimal accuracy-vs-model size cruve between E2B and E4B without any additional training. Using a simple technique called **Mix-n-Match** one can extract a model of any size between E2B and E4B from the main E4B model.

Why would you slice a model? Given your specific deployment requirements, the E2B and E4B may not be the right fit. You may want to get an E3B, for example, which has quality higher than the E2B while requiring less compute than E4B.

**In this notebook, you get to play with MatFormers and Mix-n-Match. You will specify which configuration you want for the submodel based on FFN dimension and skp layers, and then you will export the model to Hugging Face, enabling you to use with your favorite tools.**

## Setup


In [ ]:
# @title Install dependencies
# @markdown Run this cell to install all the required dependencies. In particular, you'll need Hugging Face `transformers` and `timm` versions that support Gemma 3n. Note that you may need to restart the notebook after executing the following cell.

# Install a transformers version that supports Gemma 3n (>= 4.53)
!pip install "transformers>=4.53"

In [ ]:
# @title Login to Hugging Face
# @markdown This is required so you can push the model to Hugging Face. You also need to make sure you have access to the Gemma 3n model repositories.

from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# @title Import and Export Options
# @markdown The MatFormer Lab allows you to load a Gemma 3n 4B checkpoint (either pre-trained or instruct-tuned) and to slice it. Below, please specify:

# @markdown * The original repository ID from the checkpoint in Hugging Face

# @markdown * A local path where the model will be saved

# @markdown * A name of a repository to push the new checkpoint to

original_model_id = "google/gemma-3n-E4B-it" # @param ["google/gemma-3n-E4B-it", "google/gemma-3n-E4B-pt"]
local_output_path = "my_modified_gemma_3n_model" # @param {type:"string"}
push_hf_repo_id = "username/test-submodel"  # @param {type:"string"}

### Slicing configuration

As part of the Gemma 3n release, we share optimal slicing configurations as a [Hugging Face dataset repository](https://huggingface.co/datasets/google/gemma3n-slicing-configs), although you can also explore with your own configurations below.

Each configuration specifies:
* The hidden dimensions of the FFN
* Which layers, if any, to skip
* A MMLU accuracy associated with the pre-trained checkpoints

`layer-level` and `block-level` configurations are a result of varying the hidden dimensions of the FFN either at a `layer-level` (fine-grained) or at a `block-level` (4 local + 1 global layers).

At a `layer-level`, we found that having the global layers (instead of local layers) have higher capacity helps with accuracy for the same model size.

At a `block-level`, we found that the block skipped for E2B (ie., layers 20-24) would benfit from higher capacity when not skipped and earlier blocks can work well with lower-capacity compared to the later blocks.

**We invite the community to find even better configurations that lie on the Pareto-optimal curve between E2B and E4B!**

In [ ]:
import pandas as pd

df = pd.read_csv("hf://datasets/google/gemma3n-slicing-configs/configs.csv")
df.head()

,name,# Layers,# Effective Params (B),MMLU PT accuracy,FFN Hidden Dims,Layers Skipped
0,Main model,35,3.98,62.30%,"[2_048 * 8, 2_048 * 8, 2_048 * 8, 2_048 * 8, 2...",NaN
1,Config for official E2B Model,30,1.91,50.90%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...","[20, 21, 22, 23, 24]"
2,Config for E1.96B (layer-level),30,1.96,53.40%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...","[20, 21, 22, 23, 24]"
3,Config for E2.54B (layer-level),35,2.54,55.40%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...",NaN
4,Config for E2.69B (layer-level),35,2.69,57.70%,"[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2...",NaN


Based on your deployment scenarios, you may want to pick a different config. Select below your preferred one.

In [ ]:
#@title Config details

import ast

config_name = "Config for official E2B Model"# @param ['Config for official E2B Model', 'Config for E1.96B (layer-level)', 'Config for E2.54B (layer-level)', 'Config for E2.69B (layer-level)', 'Config for E2.98B (layer-level)', 'Config for E3.18B (layer-level)', 'Config for E3.39B (layer-level)', 'Config for E3.59B (layer-level)', 'Config for E3.79B (layer-level)', 'Config for E2.49B (block-level)', 'Config for E2.73B (block-level)', 'Config for E2.98B (block-level)', 'Config for E3.24B (block-level)', 'Config for E3.49B (block-level)', 'Config for E3.79B (block-level)']

def safe_string_to_list(value):
    """
    Converts a string representation of a list into a Python list.
    - Converts NaN/missing values to an empty list [].
    - Uses eval() to handle expressions like '2_048 * 8'.
    - Safely handles non-string values by returning them as is.
    """
    # First, check if the value is missing (NaN, None, etc.)
    if isinstance(value, list):
        return value

    # Priority 2: Now that we know it's not a list, check if it's a missing value.
    if pd.isna(value):
        return []

    # Priority 3: If it's a string, try to evaluate it.
    if isinstance(value, str):
        try:
            return eval(value)
        except (SyntaxError, NameError):
            return value  # Return invalid string as is

    # Fallback for any other type (like an integer)
    return value

df['FFN Hidden Dims List'] = df['FFN Hidden Dims'].apply(safe_string_to_list)
df['Layers Skipped'] = df['Layers Skipped'].apply(safe_string_to_list)

df_indexed = df.set_index('name')
model_row = df_indexed.loc[config_name]

layers_to_skip = model_row['Layers Skipped']
ffn_hidden_dims = model_row['FFN Hidden Dims List']
ffn_hidden_dims_str = model_row['FFN Hidden Dims']

print(config_name)
print("\nLayers Skipped:")
print(layers_to_skip)
print("\nFFN Hidden Dims:")
print(ffn_hidden_dims_str)

Config for official E2B Model

Layers Skipped:
[20, 21, 22, 23, 24]

FFN Hidden Dims:
[2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4, 2_048 * 4]


If you want to set up your own configuration, please uncomment the following cell and assign values

In [ ]:
# Custom config
#
# layers_to_skip = [] # e.g. [20, 21, 22, 23, 24]
# ffn_hidden_dims = [] # e.g. [2048 * 4, ...]
# ffn_hidden_dims_str = str(ffn_hidden_dims)

## Slicing

### Load the model config and verify slicing configuration

Note: we do not load the model at this stage, just verify that the slicing configuration is possible

In [ ]:
from transformers import AutoConfig, AutoTokenizer

original_config = AutoConfig.from_pretrained(original_model_id)

config.json:   0%|          | 0.00/4.11k [00:00<?, ?B/s]

In [ ]:
model_config = original_config.text_config

num_layers = model_config.num_hidden_layers
final_num_layers = num_layers - len(layers_to_skip)

if len(ffn_hidden_dims) != final_num_layers:
    raise ValueError(
        f"The length of ffn_hidden_dims ({len(ffn_hidden_dims)}) must be equal "
        f"to the final number of layers ({final_num_layers})."
    )

### Update configuration

In [ ]:
num_kv_comp_layers = model_config.num_hidden_layers - model_config.num_kv_shared_layers
local_kv_sharing_layer_idx = num_kv_comp_layers - 2
global_kv_sharing_layer_idx = num_kv_comp_layers - 1

if (local_kv_sharing_layer_idx in layers_to_skip or global_kv_sharing_layer_idx in layers_to_skip):
  raise ValueError(f'Layers {local_kv_sharing_layer_idx} and {global_kv_sharing_layer_idx} are reserved.')

count_kv_sharing = sum(1 for layer in layers_to_skip if layer >= 20)
model_config.num_kv_shared_layers -= count_kv_sharing

In [ ]:
count_activation_sparsity = sum(1 for layer in layers_to_skip if layer <= 9)
activation_sparsity_list = [0.95] * (10 - count_activation_sparsity) + [0] * (
    final_num_layers - 10 + count_activation_sparsity
)
model_config.activation_sparsity_pattern = activation_sparsity_list

In [ ]:
model_config.num_hidden_layers = final_num_layers
model_config.intermediate_size = ffn_hidden_dims

### Save the configuration and the unchanged tokenizer

In [ ]:
original_config.save_pretrained(local_output_path)
tokenizer = AutoTokenizer.from_pretrained(original_model_id)
tokenizer.save_pretrained(local_output_path)

print(f"New config saved to {local_output_path}")
print(f"Final number of layers: {model_config.num_hidden_layers}")

tokenizer_config.json:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

New config saved to my_modified_gemma_3n_model
Final number of layers: 30


### Load the model checkpoints

Note: we are saving the model to disk, so there's no need to have a large CPU/GPU.

In [ ]:
import os

from huggingface_hub import snapshot_download

model_path = snapshot_download(original_model_id, allow_patterns=["*.safetensors"])
safetensor_files = [os.path.join(model_path, f) for f in os.listdir(model_path) if f.endswith('.safetensors')]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.66G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

### Slice the model!

In [ ]:
from safetensors import safe_open
from tqdm.auto import tqdm
import re
import torch
import gc

from safetensors.torch import save_file

kept_layers_indices = [i for i in range(num_layers) if i not in layers_to_skip]
layer_rename_map = {old_idx: new_idx for new_idx, old_idx in enumerate(kept_layers_indices)}

# This will store the mapping of tensor names to the file they are saved in
weight_map = {}

# This will store tensors for the current shard we are building
new_shard_state_dict = {}
shard_counter = 1
total_size = 0

pbar = tqdm(total=len(safetensor_files), desc="Processing shards")

for shard_path in safetensor_files:
    # Open a shard for streaming
    with safe_open(shard_path, framework="pt", device="cpu") as f:
        # Iterate over each tensor in the shard
        for tensor_name in f.keys():
            new_tensor_name = tensor_name
            tensor = f.get_tensor(tensor_name)

            # Case 1: Handle layer-specific parameters
            match = re.search(r'\.layers\.(\d+)\.', tensor_name)
            if match:
                old_layer_idx = int(match.group(1))

                # If this layer is meant to be skipped, we just continue to the next tensor
                if old_layer_idx in layers_to_skip:
                    continue

                # Get the new sequential layer index
                new_layer_idx = layer_rename_map[old_layer_idx]
                new_tensor_name = tensor_name.replace(
                    f'.layers.{old_layer_idx}.',
                    f'.layers.{new_layer_idx}.'
                )

                # Get the target FFN dimension for this new layer
                target_ffn_dim = ffn_hidden_dims[new_layer_idx]

                # Check if this parameter is part of the FFN and needs slicing
                if 'mlp.gate_proj.weight' in new_tensor_name or 'mlp.up_proj.weight' in new_tensor_name:
                    # These layers project from model_dim -> ffn_hidden_dim.
                    # We slice the output dimension (dim 0).
                    tensor = tensor[:target_ffn_dim, :].contiguous()
                elif 'mlp.down_proj.weight' in new_tensor_name:
                    # This layer projects from ffn_hidden_dim -> model_dim.
                    # We slice the input dimension (dim 1).
                    tensor = tensor[:, :target_ffn_dim].contiguous()

            # Case 2: Handle special non-layer parameters that need slicing
            elif 'per_layer_model_projection' in tensor_name:
                # Reshape, slice based on kept layers, and reshape back
                reshaped_params = tensor.reshape((num_layers, tensor.shape[0] // num_layers, tensor.shape[1]))
                tensor = reshaped_params[kept_layers_indices, :, :]
                tensor = tensor.reshape(-1, tensor.shape[-1]).contiguous()

            elif 'embed_tokens_per_layer' in tensor_name:
                # Reshape, slice based on kept layers, and reshape back
                reshaped_params = tensor.reshape((tensor.shape[0], num_layers, tensor.shape[1] // num_layers))
                tensor = reshaped_params[:, kept_layers_indices, :]
                tensor = tensor.reshape(tensor.shape[0], -1).contiguous()

            # Add the (potentially modified) tensor to the new shard
            new_shard_state_dict[new_tensor_name] = tensor

            # Check if the current shard is getting too big
            current_shard_size = sum(t.numel() * t.element_size() for t in new_shard_state_dict.values())
            if current_shard_size > 4000000000: # Create new shard if current is over 4GB
                shard_filename = f"model-{(shard_counter):05d}-of-XXXXX.safetensors"
                print(f"Saving shard {shard_filename} (size: {current_shard_size / 1e9:.2f} GB)")
                save_file(new_shard_state_dict, os.path.join(local_output_path, shard_filename), metadata={'format': 'pt'})

                # Record which tensors are in this shard
                for k in new_shard_state_dict.keys():
                    weight_map[k] = os.path.basename(shard_filename)

                # Reset for the next shard
                shard_counter += 1
                new_shard_state_dict = {}
                gc.collect() # Free up memory
    pbar.update(1)

pbar.close()

Processing shards:   0%|          | 0/4 [00:00<?, ?it/s]

Saving shard model-00001-of-XXXXX.safetensors (size: 4.02 GB)
Saving shard model-00002-of-XXXXX.safetensors (size: 4.29 GB)


In [ ]:
# Save any remaining tensors in the last shard
if new_shard_state_dict:
    shard_filename = f"model-{(shard_counter):05d}-of-XXXXX.safetensors"
    print(f"Saving final shard {shard_filename}")
    save_file(new_shard_state_dict, os.path.join(local_output_path, shard_filename), metadata={'format': 'pt'})
    for k in new_shard_state_dict.keys():
        weight_map[k] = os.path.basename(shard_filename)

Saving final shard model-00003-of-XXXXX.safetensors


In [ ]:
del new_shard_state_dict
gc.collect()

300

In [ ]:
import json
print("\n--- 3. Finalizing Model Save ---")

# The total number of shards we created
num_shards = shard_counter

# Update the "XXXXX" in the filenames to the correct total number of shards
for i in range(1, num_shards + 1):
    old_filename = f"model-{(i):05d}-of-XXXXX.safetensors"
    new_filename = f"model-{(i):05d}-of-{(num_shards):05d}.safetensors"

    # Rename the file
    os.rename(os.path.join(local_output_path, old_filename), os.path.join(local_output_path, new_filename))

    # Update the weight_map to point to the new filename
    for k, v in weight_map.items():
        if v == old_filename:
            weight_map[k] = new_filename

# Create and save the index.json file
index_json = {
    "metadata": {
        "total_size": sum(os.path.getsize(os.path.join(local_output_path, f)) for f in os.listdir(local_output_path) if f.endswith('.safetensors'))
    },
    "weight_map": weight_map
}

with open(os.path.join(local_output_path, "model.safetensors.index.json"), "w") as f:
    json.dump(index_json, f, indent=2)

print(f"\n✅ Model slicing complete. New model saved in: {local_output_path}")


--- 3. Finalizing Model Save ---

✅ Model slicing complete. New model saved in: my_modified_gemma_3n_model


In [ ]:
from huggingface_hub import  ModelCard, ModelCardData

card = ModelCard.load(original_model_id)
card.data.base_model = original_model_id
del card.data.extra_gated_heading
del card.data.extra_gated_prompt
card.data.tags.append("matformer")

new_description = f"""
> [!Note]
> This is a submodel derived from `{original_model_id}`. It has been modified by slicing specific layers and resizing FFN dimensions. It is not the original model.
> To learn more about MatFormers, please review the [launch blog](https://developers.googleblog.com/en/introducing-gemma-3n-developer-guide) and generate your own submodels
with the [MatFormer Lab](https://goo.gle/gemma3n-matformer-lab).
>

Skipped layers: {layers_to_skip}

FFN hidden dimensions: {ffn_hidden_dims_str}
"""

card.text = new_description + "\n" + card.text
print("Prepended custom description to the model card content.")

new_readme_path = os.path.join(local_output_path, "README.md")
card.save(new_readme_path)
print(f"New README.md saved to '{new_readme_path}'")

README.md:   0%|          | 0.00/23.4k [00:00<?, ?B/s]

Prepended custom description to the model card content.
New README.md saved to 'my_modified_gemma_3n_model/README.md'


## Push the model to Hugging Face

In [ ]:
from huggingface_hub import HfApi

print(f"Creating private repository: {push_hf_repo_id}")

# Instantiate the HfApi client
api = HfApi()

# Create a new private repository on the Hub.
repo_url = api.create_repo(
    repo_id=push_hf_repo_id,
    private=True,
    exist_ok=True
)

In [ ]:
print(f"Uploading files from '{local_output_path}' to '{push_hf_repo_id}'...")
api.upload_folder(
    folder_path=local_output_path,
    repo_id=push_hf_repo_id,
    repo_type="model",
    commit_message="Upload sliced model checkpoint"
)

In [ ]:
#@title Verify new model can be loaded


from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(push_hf_repo_id, torch_dtype=torch.bfloat16, device_map="auto")

print(f"Total Parameters: {model.num_parameters():,}") # 5,976,833,408
print(f"Total Text Parameters: {model.language_model.num_parameters():,}") # 4,456,156,768
print(f"Effective Parameters (excluding vision, audio, and Per-Layer-Embeddings): {model.language_model.num_parameters(exclude_embeddings=True):,}") # 1,905,495,648

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Total Parameters: 5,976,833,408
Total Text Parameters: 4,456,156,768
Effective Parameters (excluding vision, audio, and Per-Layer-Embeddings): 1,905,495,648


## Citation

```
@article{gemma_3n_2025,
    title={Gemma 3n MatFormer Lab},
    url={Add URL,},
    publisher={Google DeepMind},
    author={Puranjay Datta, Aditya Kusupati, Ryan Mullins, Omar Sanseviero, Rakesh Shivanna, Gemma Team},
    year={2025}
}
```